In [3]:
import json,urllib.request
import pandas as pd
from scipy.stats import zscore
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw
from scipy.fftpack import fft, ifft
from saxpy import sax
from saxpy.sax import sax_via_window
from scipy import spatial
from sklearn.metrics.pairwise import cosine_similarity
import numpy

In [4]:
stockSymbol="MSFT"
#print("https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=",stockSymbol,"&outputsize=full&apikey=GZ78DWYK4UQOVQ83")
data = urllib.request.urlopen("https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=MSFT&outputsize=full&apikey=GZ78DWYK4UQOVQ83").read()
output = json.loads(data)
jsonOutput=output["Time Series (Daily)"]
dF_Daily=pd.DataFrame.from_dict(jsonOutput, orient="index")
dF_Daily.index=pd.to_datetime(dF_Daily.index)

In [5]:
dataframes=[];
for x in range(1998,2018):
    dataframes.append(dF_Daily[dF_Daily.index.year == x])
min=365;
for x in range(0,len(dataframes)):
    if(len(dataframes[x])<min):
        min=len(dataframes[x])
combinedYearsDataFrame=pd.DataFrame();
for x in range(0,len(dataframes)):
    dataframes[x]=dataframes[x][:min]
    #print(dataframes[x])
    dataframe = [ float(x) for x in list(dataframes[x]["1. open"]) ]
    combinedYearsDataFrame[1998+x]=dataframe

In [6]:
combinedYearsDataFrame

,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,129.63,139.61,117.37,44.13,66.65,52.30,27.580,26.8000,26.250,29.91,35.790,19.5328,30.62,28.0500,26.5500,27.2500,37.350,46.66,54.32,62.790
1,131.25,141.88,113.56,43.19,67.11,53.59,27.730,26.8600,26.770,29.70,35.220,20.2000,30.85,27.9400,26.8199,27.6300,37.200,46.37,54.93,62.480
2,129.75,149.50,111.12,47.81,69.25,54.02,28.190,26.8400,26.930,29.63,35.190,20.7500,30.88,27.9000,27.3800,27.2700,36.850,46.38,54.32,62.190
3,129.88,149.75,112.19,48.50,69.75,54.92,28.170,26.8600,26.880,29.65,34.550,20.1900,30.63,28.0400,27.5300,26.7700,36.325,45.98,52.70,62.300
4,128.63,152.19,108.62,48.94,68.69,55.37,28.390,26.8300,26.930,30.00,34.710,19.6300,30.28,28.6400,28.0500,26.7500,36.000,46.75,52.37,62.760
5,130.06,150.88,113.44,50.00,69.72,54.72,28.030,26.6000,26.650,29.80,33.360,20.1700,30.71,28.2600,27.9300,26.7200,35.880,47.61,52.51,62.730
6,124.62,148.13,111.50,51.00,68.48,55.10,27.670,26.6900,27.010,29.76,34.350,19.7100,30.15,28.2000,27.4300,26.6500,35.900,47.42,52.76,62.610
7,129.50,136.00,108.50,53.00,69.52,56.52,27.540,26.7700,27.250,30.65,34.140,19.5200,30.26,28.1200,27.8700,26.4900,35.990,46.97,53.80,63.060
8,132.13,145.25,104.37,54.88,68.30,56.33,27.510,26.6900,27.030,31.26,34.460,19.5300,30.31,28.3300,27.9300,26.9000,34.730,45.96,52.00,62.620
9,130.38,142.94,107.19,53.38,68.66,57.00,27.550,26.4000,26.900,31.26,34.030,19.0900,31.08,28.0800,28.4000,26.8300,35.900,46.22,51.31,62.680


## DTW

In [7]:
for x in range(1998,2018):
    combinedYearsDataFrame[x]=zscore(combinedYearsDataFrame[x])


#Keeping a threshold of -3 and +3 for a +! and others for a -1
pathDict={};
for x in range(1998,2018):
    for y in range(x, 2018):
        distance, path = fastdtw(combinedYearsDataFrame[x], combinedYearsDataFrame[y], dist=euclidean)
        #print(distance)
        score=0
        for z in range(0,len(path)):
            #print(path[z][1])
            if((path[z][0]-path[z][1]<=15) &(path[z][0]-path[z][1]>=-15)):
                score=score+1;
            else:
                score=score-1;
        pathDict[x,y]=score

In [8]:
pathDict

{(1998, 1998): 248,
 (1998, 1999): 72,
 (1998, 2000): 22,
 (1998, 2001): 74,
 (1998, 2002): -261,
 (1998, 2003): -53,
 (1998, 2004): 103,
 (1998, 2005): 33,
 (1998, 2006): -241,
 (1998, 2007): -110,
 (1998, 2008): -89,
 (1998, 2009): -135,
 (1998, 2010): -303,
 (1998, 2011): 170,
 (1998, 2012): -30,
 (1998, 2013): -56,
 (1998, 2014): -169,
 (1998, 2015): -256,
 (1998, 2016): -116,
 (1998, 2017): -144,
 (1999, 1999): 248,
 (1999, 2000): -25,
 (1999, 2001): -268,
 (1999, 2002): -94,
 (1999, 2003): -127,
 (1999, 2004): -234,
 (1999, 2005): -236,
 (1999, 2006): 31,
 (1999, 2007): -145,
 (1999, 2008): -83,
 (1999, 2009): -309,
 (1999, 2010): -169,
 (1999, 2011): 250,
 (1999, 2012): -90,
 (1999, 2013): -137,
 (1999, 2014): -121,
 (1999, 2015): -220,
 (1999, 2016): -10,
 (1999, 2017): -144,
 (2000, 2000): 248,
 (2000, 2001): -49,
 (2000, 2002): 155,
 (2000, 2003): -116,
 (2000, 2004): -121,
 (2000, 2005): -100,
 (2000, 2006): -205,
 (2000, 2007): -139,
 (2000, 2008): -162,
 (2000, 2009): 283,

## Symbolic Aggregate approXimation

In [9]:
# All tests will be run with 6 letter words
# and 5 letter alphabet
#sax1=sax_via_window(combinedYearsDataFrame[1998], 2, 2)
#sax1

In [10]:
#sax2=sax_via_window(combinedYearsDataFrame[1999], 2, 2)
#sax2

In [11]:
cluster=[]
for key in pathDict:
    print(key)
    if(key[0]!=key[1]):
        if(pathDict[key]>0):
            cluster.append([key[0],key[1]])

(1998, 1998)
(1998, 1999)
(1998, 2000)
(1998, 2001)
(1998, 2002)
(1998, 2003)
(1998, 2004)
(1998, 2005)
(1998, 2006)
(1998, 2007)
(1998, 2008)
(1998, 2009)
(1998, 2010)
(1998, 2011)
(1998, 2012)
(1998, 2013)
(1998, 2014)
(1998, 2015)
(1998, 2016)
(1998, 2017)
(1999, 1999)
(1999, 2000)
(1999, 2001)
(1999, 2002)
(1999, 2003)
(1999, 2004)
(1999, 2005)
(1999, 2006)
(1999, 2007)
(1999, 2008)
(1999, 2009)
(1999, 2010)
(1999, 2011)
(1999, 2012)
(1999, 2013)
(1999, 2014)
(1999, 2015)
(1999, 2016)
(1999, 2017)
(2000, 2000)
(2000, 2001)
(2000, 2002)
(2000, 2003)
(2000, 2004)
(2000, 2005)
(2000, 2006)
(2000, 2007)
(2000, 2008)
(2000, 2009)
(2000, 2010)
(2000, 2011)
(2000, 2012)
(2000, 2013)
(2000, 2014)
(2000, 2015)
(2000, 2016)
(2000, 2017)
(2001, 2001)
(2001, 2002)
(2001, 2003)
(2001, 2004)
(2001, 2005)
(2001, 2006)
(2001, 2007)
(2001, 2008)
(2001, 2009)
(2001, 2010)
(2001, 2011)
(2001, 2012)
(2001, 2013)
(2001, 2014)
(2001, 2015)
(2001, 2016)
(2001, 2017)
(2002, 2002)
(2002, 2003)
(2002, 2004)

In [12]:
tempDF=pd.DataFrame(cluster,columns=['first','second'])

In [13]:
tempDF

,first,second
0,1998,1999
1,1998,2000
2,1998,2001
3,1998,2004
4,1998,2005
5,1998,2011
6,1999,2006
7,1999,2011
8,2000,2002
9,2000,2009


In [14]:
InCluster=set();
clust=[]
for x in range(1998,2017):
    clust.append([]);
    currArr=x-1998;
    try:
        temp=tempDF.groupby("first").get_group(x)
        temp_cluster=list(temp["second"])
        print(currArr,temp_cluster)
        for i in range(0,len(temp_cluster)):
            if(x not in InCluster):
                clust[currArr].append(x)
            if(temp_cluster[i] not in InCluster):
                clust[currArr].append(temp_cluster[i])
            InCluster.add(x)
            InCluster.add(temp_cluster[i])
    except:
        print("Not a group")
    

0 [1999, 2000, 2001, 2004, 2005, 2011]
1 [2006, 2011]
2 [2002, 2009, 2010, 2011, 2013, 2014, 2016, 2017]
3 [2007]
Not a group
5 [2006, 2008, 2011, 2014, 2016]
6 [2009, 2011, 2013, 2014, 2015, 2017]
Not a group
8 [2011, 2012, 2016]
9 [2013, 2014, 2015, 2017]
10 [2009, 2012, 2013, 2014, 2015, 2016, 2017]
11 [2013, 2014, 2015, 2017]
Not a group
13 [2013, 2016, 2017]
Not a group
15 [2015, 2017]
16 [2015, 2016, 2017]
Not a group
18 [2017]


In [15]:
final_clusts = [x for x in clust if x != []]

In [16]:
final_clusts

[[1998, 1999, 2000, 2001, 2004, 2005, 2011],
 [2006],
 [2002, 2009, 2010, 2013, 2014, 2016, 2017],
 [2007],
 [2003, 2008],
 [2015],
 [2012]]

## COSINE SIMILARITY:

In [17]:
cosineMatrix=[]
for x in range(1998,2018):
    cosineMatrix.append([]);
    for y in range(1998,2018):
        result = 1 - spatial.distance.cosine(combinedYearsDataFrame[x], combinedYearsDataFrame[y])
        cosineMatrix[x-1998].append(result);

In [18]:
cosineMatrix

[[1.0,
  0.37751590327724416,
  0.19080803599650564,
  -0.064600772579080479,
  0.28833912526325545,
  0.65673404475561969,
  0.42668665419266127,
  0.50724798607320987,
  0.35928137296869522,
  0.34239945915606551,
  -0.08444603021415098,
  0.23201746616419916,
  0.013151786347041727,
  0.57253814798686431,
  -0.49013256857411847,
  -0.047369003343376459,
  0.018784528199568107,
  0.22291146030158226,
  0.2202709439766537,
  0.13235784139973483],
 [0.37751590327724416,
  1.0,
  0.75141664411527176,
  -0.49732720409495101,
  0.73546945244465767,
  0.58473497514356743,
  -0.13576580934472426,
  -0.055553137478996373,
  0.30896506859095374,
  -0.2274625254378666,
  0.39871554263855602,
  -0.54766765039240561,
  0.50436837940121693,
  0.52410880100270507,
  0.15355071799201636,
  -0.70549410730954332,
  -0.59196630408674311,
  -0.33809360578443259,
  -0.27029487589103418,
  -0.51614111053009637],
 [0.19080803599650564,
  0.75141664411527176,
  1.0,
  -0.44337646014078347,
  0.773452808467

In [19]:
cosineMatrixDF=pd.DataFrame(cosineMatrix)
for x in range(0,len(cosineMatrixDF)):
    cosineMatrixDF.rename(columns={x:1998+x}, 
                 inplace=True)
tempList=list(cosineMatrixDF.columns)
cosineMatrixDF["year"]=tempList
cosineMatrixDF=cosineMatrixDF.set_index("year")

In [20]:
cosineMatrixDF

,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
year,,,,,,,,,,,,,,,,,,,,
1998,1.000000,0.377516,0.190808,-0.064601,0.288339,0.656734,0.426687,0.507248,0.359281,0.342399,-0.084446,0.232017,0.013152,0.572538,-0.490133,-0.047369,0.018785,0.222911,0.220271,0.132358
1999,0.377516,1.000000,0.751417,-0.497327,0.735469,0.584735,-0.135766,-0.055553,0.308965,-0.227463,0.398716,-0.547668,0.504368,0.524109,0.153551,-0.705494,-0.591966,-0.338094,-0.270295,-0.516141
2000,0.190808,0.751417,1.000000,-0.443376,0.773453,0.575275,-0.224054,-0.189558,-0.021136,-0.457442,0.695232,-0.751774,0.579343,0.427548,0.324765,-0.833524,-0.766940,-0.562466,-0.548131,-0.786953
2001,-0.064601,-0.497327,-0.443376,1.000000,-0.289058,-0.374897,0.096502,0.100385,-0.428363,0.380773,-0.279117,0.252800,-0.206959,-0.356433,-0.216211,0.574532,0.167814,0.407734,-0.089675,0.297766
2002,0.288339,0.735469,0.773453,-0.289058,1.000000,0.581816,-0.224220,0.011776,0.276080,-0.013167,0.424985,-0.487142,0.691423,0.361502,-0.063047,-0.529820,-0.622609,-0.112537,-0.335297,-0.481634
2003,0.656734,0.584735,0.575275,-0.374897,0.581816,1.000000,0.280220,0.182264,0.151946,0.005450,0.510625,-0.260123,0.344303,0.691706,-0.257975,-0.497407,-0.477215,-0.134487,-0.223864,-0.385350
2004,0.426687,-0.135766,-0.224054,0.096502,-0.224220,0.280220,1.000000,0.300166,0.035440,0.360297,-0.205230,0.483585,-0.444584,0.428862,-0.418771,0.293092,0.342271,0.320874,0.235951,0.376398
2005,0.507248,-0.055553,-0.189558,0.100385,0.011776,0.182264,0.300166,1.000000,0.284351,0.451927,-0.368840,0.526905,-0.276769,0.046648,-0.631835,0.354111,0.438120,0.514595,0.512352,0.490209
2006,0.359281,0.308965,-0.021136,-0.428363,0.276080,0.151946,0.035440,0.284351,1.000000,0.410621,-0.445641,0.339014,0.216819,0.312213,-0.314035,0.024582,0.319004,0.361224,0.653099,0.409377


In [21]:
cosineHashSet=set();
threshold=0.5;
cosCluster=[]
for x in range(1998,2018):
    cosCluster.append([]);
    currList=list(cosineMatrixDF[cosineMatrixDF[x]>0.5].index)
    for i in range(0,len(currList)):
        if(currList[i] not in cosineHashSet):
            cosCluster[x-1998].append(currList[i])
        cosineHashSet.add(x)
        cosineHashSet.add(currList[i])

In [22]:
final_cosClusts = [x for x in cosCluster if x != []]

In [23]:
final_cosClusts

[[1998, 2003, 2005, 2011],
 [1999, 2000, 2002, 2010],
 [2008],
 [2001, 2013],
 [2004],
 [2009, 2015, 2016],
 [2006],
 [2007, 2014, 2017],
 [2012]]

In [24]:
data = {}
for x in range(0, len(final_cosClusts)):
    key="Cluster"+str(x+1)
    data[key] = list(map(int, final_cosClusts[x]))
json_data = json.dumps(data)
json_data

'{"Cluster1": [1998, 2003, 2005, 2011], "Cluster2": [1999, 2000, 2002, 2010], "Cluster3": [2008], "Cluster4": [2001, 2013], "Cluster5": [2004], "Cluster6": [2009, 2015, 2016], "Cluster7": [2006], "Cluster8": [2007, 2014, 2017], "Cluster9": [2012]}'

In [25]:
type(final_cosClusts[x][0])

numpy.int64

In [26]:
list(cosineMatrixDF[1998])

[1.0,
 0.37751590327724416,
 0.19080803599650564,
 -0.064600772579080479,
 0.28833912526325545,
 0.65673404475561969,
 0.42668665419266127,
 0.50724798607320987,
 0.35928137296869522,
 0.34239945915606551,
 -0.08444603021415098,
 0.23201746616419916,
 0.013151786347041727,
 0.57253814798686431,
 -0.49013256857411847,
 -0.047369003343376459,
 0.018784528199568107,
 0.22291146030158226,
 0.2202709439766537,
 0.13235784139973483]

In [27]:
[1.0,0.59,0.67,4.0]

[1.0, 0.59, 0.67, 4.0]

In [29]:
import saxpy